## Data Cleaning - Pre-Generated Data Files  
### EPA - United States Environmental Protection Agency  

**Link:** [EPA Air Data Download Files](https://aqs.epa.gov/aqsweb/airdata/download_files.html)


In [29]:
import pandas as pd

In [30]:
import pandas as pd

def cleanData(year: str):
    # change to respective path
    df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_{year}.csv', low_memory=False)
    
    all_cols = df.columns.tolist()
    selected_cols = ['Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration', 
                     'Pollutant Standard', 'Metric Used', 'Method Name', 'Year', 
                     'Units of Measure', 'Observation Count', 'Observation Percent', 
                     'Arithmetic Mean', 'Arithmetic Standard Dev', 'Local Site Name', 
                     'Address', 'State Name', 'County Name', 'City Name']
    
    remove_cols = list(set(all_cols) - set(selected_cols))
    df.drop(remove_cols, axis=1, inplace=True)
    
    parameter_list = ["Ozone", "Sulfur Dioxide", "Carbon monoxide", "Nitrogen Dioxide"]
    df = df[df["Parameter Name"].isin(parameter_list)]  # Exclude unwanted parameters

    # change to respective path
    df.to_csv(f'/Users/rahib/ctp_data/filtered_data_{year}.csv', index=False)



In [31]:
#Define the years as strings
years = ["2023"]

In [32]:
#Clean all the data.
for year in years:
    cleanData(year)